Capstone Project - The Battle of the Neighborhoods (Week 2)
--
IBM Data Science Capstone Project
--

Table of contents
--
>Introduction: Business Problem

> Required Data

>Methodology

>Results and Discussion

>Conclusion

•	Introduction:-
--
The project aims to identify venues in Toronto, Canada based on their latitude and Longitude. In this notebook, we will identify various venues in the city of Toronto, Canada using Foursquare API to help property developers to select the location that suits them the best. So, to decide which location is suitable for the shopping mall we required to identify factors such as location cost, it will be profitable or not and so many. 
                     

•	Business Problem: -
--
The objective of this IBM data science capstone project is to analyze and select the best location for opening a shopping mall in the city of Toronto, Canada. So, for that, we are using the data science methodology and machine learning technique such as clustering, this project provides a solution to the business question.

Business question: -In the city of Toronto, Canada if a property developer is lolling to open a new shopping mall, where would you recommend that they should open it.



• Data Required :
--
>list of neighborhoods in city of Toronto

>Longitude and latitude coordinates of the neighborhood.

>Venue data related to shopping malls

In [1]:
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [3]:
# create three lists to store table data
postalCode= []
borough = []
neighborhood = []

In [4]:
# append the data into the respective lists
for rows in soup.find('table').find_all('tr'):
    cells = rows.find_all('td')
    if(len(cells) > 0):
        postalCode.append(cells[0].text.replace("\n",""))
        borough.append(cells[1].text.replace("\n",""))
        neighborhood.append(cells[2].text.rstrip('\n'))

In [5]:
# create a new DataFrame from the three lists
df = pd.DataFrame({"PostalCode": postalCode,"Borough": borough,"Neighborhood": neighborhood})

df.head()

,Borough,Neighborhood,PostalCode
0,Not assigned,,M1A
1,Not assigned,,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Regent Park / Harbourfront,M5A


In [6]:
df1 = df[df.Borough != 'Not assigned']
df1.head(10)

,Borough,Neighborhood,PostalCode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Regent Park / Harbourfront,M5A
5,North York,Lawrence Manor / Lawrence Heights,M6A
6,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A
8,Etobicoke,Islington Avenue,M9A
9,Scarborough,Malvern / Rouge,M1B
11,North York,Don Mills,M3B
12,East York,Parkview Hill / Woodbine Gardens,M4B
13,Downtown Toronto,"Garden District, Ryerson",M5B


In [7]:
df2=df1.reset_index(drop=True)
df2.head(10)

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Regent Park / Harbourfront,M5A
3,North York,Lawrence Manor / Lawrence Heights,M6A
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,Malvern / Rouge,M1B
7,North York,Don Mills,M3B
8,East York,Parkview Hill / Woodbine Gardens,M4B
9,Downtown Toronto,"Garden District, Ryerson",M5B


In [8]:
df1.shape

(103, 3)

In [9]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df3 = pd.merge(df2, coordinates, on='PostalCode')
df3.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,Regent Park / Harbourfront,M5A,43.654260,-79.360636
3,North York,Lawrence Manor / Lawrence Heights,M6A,43.718518,-79.464763
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A,43.662301,-79.389494


Use geopy library to get the latitude and longitude values of Toronto.
--

In [12]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods.
--

In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    

map_toronto

Define Foursquare Credentials and Version
--

In [36]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = 'Foursquare API version' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


In [39]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    long,
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=your Foursquare ID&client_secret=your Foursquare Secret&v=Foursquare API version&ll=43.7942003,-79.52099940000001&radius=2000&limit=100'

Now, let's get the top 100 venues that are in Toronto within a radius of 2000 meters.
--

In [16]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [17]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(8638, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.753259,-79.329656,Island Foods,43.745866,-79.346035,Caribbean Restaurant
4,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket
5,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
6,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space
7,Parkwoods,43.753259,-79.329656,The Captain's Boil,43.754986,-79.349524,Seafood Restaurant
8,Parkwoods,43.753259,-79.329656,Me Va Me Kitchen Express,43.754957,-79.351894,Mediterranean Restaurant
9,Parkwoods,43.753259,-79.329656,Menchie's,43.754764,-79.350013,Frozen Yogurt Shop


Let's check how many venues were returned for each PostalCode
--

In [18]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Agincourt,100,100,100,100,100,100
Alderwood / Long Branch,100,100,100,100,100,100
Bathurst Manor / Wilson Heights / Downsview North,57,57,57,57,57,57
Bayview Village,52,52,52,52,52,52
Bedford Park / Lawrence Manor East,100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
Birch Cliff / Cliffside West,45,45,45,45,45,45
Brockton / Parkdale Village / Exhibition Place,100,100,100,100,100,100
Business reply mail Processing CentrE,100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 333 uniques categories.


In [20]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Caribbean Restaurant', 'Golf Course', 'Park', 'Supermarket',
       'Liquor Store', 'Event Space', 'Seafood Restaurant',
       'Mediterranean Restaurant', 'Frozen Yogurt Shop', 'Café',
       'Gym / Fitness Center', 'Asian Restaurant',
       'Middle Eastern Restaurant', 'Japanese Restaurant', 'Pool Hall',
       'American Restaurant', 'Fast Food Restaurant', 'Steakhouse',
       'Ice Cream Shop', 'Discount Store', 'Coffee Shop',
       'Persian Restaurant', 'Salad Place', 'Grocery Store', 'Pizza Place',
       'Gym', 'Italian Restaurant', 'Hakka Restaurant',
       'Fried Chicken Joint', 'Chinese Restaurant',
       'Paper / Office Supplies Store', 'Burger Joint', 'Sandwich Place',
       'Thai Restaurant', 'Bank', 'Hotel', 'Fish & Chips Shop', 'Pharmacy',
       'Beer Store', 'Gas Station', 'Greek Restaurant', 'Sushi Restaurant',
       'Automotive Shop', 'Breakfast Spot', 'Toy / Game Store',
       'Furniture / Home Store', 'Bus Stop', 'Wings Joint', 'Skating Rink',
       

In [21]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

Analyze Each Neighborhood
--

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(8638, 334)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Sou

In [23]:
toronto_grouped = toronto_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(98, 334)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Sou

In [24]:
len(toronto_grouped[toronto_grouped["Shopping Mall"] > 0])

43

In [25]:
toronto_mall = toronto_grouped[["Neighborhoods","Shopping Mall"]]

In [26]:
toronto_mall.head()

,Neighborhoods,Shopping Mall
0,Agincourt,0.020000
1,Alderwood / Long Branch,0.010000
2,Bathurst Manor / Wilson Heights / Downsview North,0.017544
3,Bayview Village,0.019231
4,Bedford Park / Lawrence Manor East,0.000000


Cluster Neighborhoods
--

Run k-means to cluster the neighborhood into 3 clusters.

In [27]:
# set number of clusters
kclusters = 3

toronto_clustering = toronto_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 2, 0, 1, 0, 0, 0, 0])

In [28]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_mall.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

In [29]:
toronto_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
toronto_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Agincourt,0.020000,2
1,Alderwood / Long Branch,0.010000,1
2,Bathurst Manor / Wilson Heights / Downsview North,0.017544,2
3,Bayview Village,0.019231,2
4,Bedford Park / Lawrence Manor East,0.000000,0


In [30]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(df3.set_index("Neighborhood"), on="Neighborhood")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(103, 7)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,PostalCode,Latitude,Longitude
0,Agincourt,0.020000,2,Scarborough,M1S,43.794200,-79.262029
1,Alderwood / Long Branch,0.010000,1,Etobicoke,M8W,43.602414,-79.543484
2,Bathurst Manor / Wilson Heights / Downsview North,0.017544,2,North York,M3H,43.754328,-79.442259
3,Bayview Village,0.019231,2,North York,M2K,43.786947,-79.385975
4,Bedford Park / Lawrence Manor East,0.000000,0,North York,M5M,43.733283,-79.419750


In [31]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(103, 7)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,PostalCode,Latitude,Longitude
47,Lawrence Park,0.000000,0,Central Toronto,M4N,43.728020,-79.388790
57,Northwest,0.000000,0,Etobicoke,M9W,43.706748,-79.594054
56,North Toronto West,0.000000,0,Central Toronto,M4R,43.715383,-79.405678
55,North Park / Maple Leaf Park / Upwood Park,0.000000,0,North York,M6L,43.713756,-79.490074
54,New Toronto / Mimico South / Humber Bay Shores,0.000000,0,Etobicoke,M8V,43.605647,-79.501321
53,Moore Park / Summerhill East,0.000000,0,Central Toronto,M4T,43.689574,-79.383160
50,Malvern / Rouge,0.000000,0,Scarborough,M1B,43.806686,-79.194353
49,Little Portugal / Trinity,0.000000,0,West Toronto,M6J,43.647927,-79.419750
48,Leaside,0.000000,0,East York,M4G,43.709060,-79.363452
44,Kensington Market / Chinatown / Grange Park,0.000000,0,Downtown Toronto,M5T,43.653206,-79.400049


In [32]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters
--

Cluster 0
--

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Borough,PostalCode,Latitude,Longitude
47,Lawrence Park,0.000000,0,Central Toronto,M4N,43.728020,-79.388790
57,Northwest,0.000000,0,Etobicoke,M9W,43.706748,-79.594054
56,North Toronto West,0.000000,0,Central Toronto,M4R,43.715383,-79.405678
55,North Park / Maple Leaf Park / Upwood Park,0.000000,0,North York,M6L,43.713756,-79.490074
54,New Toronto / Mimico South / Humber Bay Shores,0.000000,0,Etobicoke,M8V,43.605647,-79.501321
53,Moore Park / Summerhill East,0.000000,0,Central Toronto,M4T,43.689574,-79.383160
50,Malvern / Rouge,0.000000,0,Scarborough,M1B,43.806686,-79.194353
49,Little Portugal / Trinity,0.000000,0,West Toronto,M6J,43.647927,-79.419750
48,Leaside,0.000000,0,East York,M4G,43.709060,-79.363452
44,Kensington Market / Chinatown / Grange Park,0.000000,0,Downtown Toronto,M5T,43.653206,-79.400049


Cluster 1
--

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Borough,PostalCode,Latitude,Longitude
32,Glencairn,0.010000,1,North York,M6B,43.709577,-79.445073
76,Stn A PO Boxes,0.010000,1,Downtown Toronto,M5W,43.646435,-79.374846
75,Steeles West / L'Amoreaux West,0.012658,1,Scarborough,M1W,43.799525,-79.318389
79,The Annex / North Midtown / Yorkville,0.010000,1,Central Toronto,M5R,43.672710,-79.405678
78,Summerhill West / Rathnelly / South Hill / For...,0.010000,1,Central Toronto,M4V,43.686412,-79.400049
84,Toronto Dominion Centre / Design Exchange,0.010000,1,Downtown Toronto,M5K,43.647177,-79.381576
73,St. James Town,0.010000,1,Downtown Toronto,M5C,43.651494,-79.375418
85,University of Toronto / Harbord,0.010000,1,Downtown Toronto,M5S,43.662696,-79.400049
87,Victoria Village,0.010204,1,North York,M4A,43.725882,-79.315572
5,Berczy Park,0.010000,1,Downtown Toronto,M5E,43.644771,-79.373306


Cluster 2
--

In [35]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Borough,PostalCode,Latitude,Longitude
96,York Mills / Silver Hills,0.020000,2,North York,M2L,43.757490,-79.374714
63,Queen's Park / Ontario Provincial Government,0.020000,2,Downtown Toronto,M7A,43.662301,-79.389494
89,Westmount,0.017857,2,Etobicoke,M9P,43.696319,-79.532242
42,Islington Avenue,0.039216,2,Etobicoke,M9A,43.667856,-79.532242
3,Bayview Village,0.019231,2,North York,M2K,43.786947,-79.385975
15,Church and Wellesley,0.020000,2,Downtown Toronto,M4Y,43.665860,-79.383160
2,Bathurst Manor / Wilson Heights / Downsview North,0.017544,2,North York,M3H,43.754328,-79.442259
0,Agincourt,0.020000,2,Scarborough,M1S,43.794200,-79.262029


•	Conclusion:
--

In this project, we've seen the method of identifying the business problem, data extraction, data preparation, and performing machine learning techniques like clustering and transform the information into three clusters based on their similarities and providing recommendations to the relevant property developer regarding the best location to open a mall. Also, the project recommended the property developer to open a new mall in cluster zero to get a maximum amount of profit with less amount of competition.
